### AnADAMA2 Example: Shotgun workflow

[AnADAMA2](http://huttenhower.sph.harvard.edu/anadama2) is the next generation of AnADAMA (Another Automated Data Analysis Management Application). AnADAMA is a tool to create reproducible workflows and execute them efficiently. Tasks can be run locally or in a grid computing environment to increase efficiency. Essential information from all tasks is recorded, using the default logger and command line reporters, to ensure reproducibility. A auto-doc feature allows for workflows to generate documentation automatically to further ensure reproducibility by capturing the latest essential workflow information. AnADAMA2 was architected to be modular allowing users to customize the application by subclassing the base grid meta-schedulers, reporters, and tracked objects (ie files, executables, etc).

* For additional information, see the [AnADAMA2 User Manual](https://bitbucket.org/biobakery/anadama2) or the [AnADAMA2 Tutorial](https://bitbucket.org/biobakery/biobakery/wiki/anadama2).
* For more example workflows, download the AnADAMA2 software source and demos ( [anadama2.tar.gz](https://pypi.python.org/pypi/anadama2) ).
* Please direct questions to the [AnADAMA Google Group](https://groups.google.com/forum/#!forum/anadama-users).
                                                        
**This example shows a workflow for shotgun data processing.**


**Step 1:** Import the workflow from anadama2. 

In [1]:
from anadama2 import Workflow

**Step 2:** Create a workflow instance. 
Since we are using Jupyter we need to turn off the command line interface for the workflow. 
The command line interface is helpful when executing a workflow directly from the command line. 
It allows the user to provide options like input/output folders at run-time. 

In [2]:
workflow = Workflow(cli=False)

**Step 3:** Set variables for the input, output, and database folders. If executing a workflow directly
you can provide these variables on the command line with the options ``--input <DIR>`` and ``--output <DIR>``. 
Since we have turned off the command line interface for Jupyter, we set these variables in a code block. 
In this code block we also search for the input files in the input folder based on the input extension. 
Additionally, we create a set of output file basenames based on the input file names.

* Demo input files can be downloaded from the [StrainPhlAn tutorial](https://bitbucket.org/biobakery/biobakery/wiki/strainphlan). All six input files need to be used for the StrainPhlAn tasks to create a tree. It is not enough to use a single sample.
* The StrainPhlAn database files can be downloaded from 
[s__Eubacterium_siraeum.markers.fasta](https://bitbucket.org/biobakery/biobakery/raw/tip/demos/biobakery_demos/data/strainphlan/input/s__Eubacterium_siraeum.markers.fasta)
and [GCF_000154325.fna.bz2](https://bitbucket.org/biobakery/biobakery/raw/tip/demos/biobakery_demos/data/strainphlan/input/GCF_000154325.fna.bz2).
* The PanPhlAn database files for E. siraeum can be downloaded from [panphlan_esiraeum16.zip](https://www.dropbox.com/sh/rfwb4i9m8s40iba/AAAgqF9RUDOJljlr49AAzYUoa/panphlan_esiraeum16.zip?dl=0).

In [3]:
import os

# set the input and output folders
input_folder = "input"
output_folder = "output"

# get all of the input files with the expected extension
input_extension = "fasta"
input_files = [os.path.join(input_folder,file) for file in os.listdir(input_folder) if file.endswith("."+input_extension)]
output_file_basenames = [os.path.join(output_folder, os.path.basename(file).replace("."+input_extension,"")) for file in input_files]

# set the folders for database files
strainphlan_db_folder = "databases/strainphlan_db"
panphlan_db_folder = "databases/panphlan_db"
panphlan_species="esiraeum16"

# get the paths to the database files needed by the workflow
strainphlan_e_siraeum_reference=os.path.join(strainphlan_db_folder,"GCF_000154325.fna.bz2")
strainphlan_e_siraeum_markers=os.path.join(strainphlan_db_folder,"s__Eubacterium_siraeum.markers.fasta")

**Step 4:** Add tasks to run MetaPhlAn2 on all of the files in the input folder. 
[MetaPhlAn2](http://huttenhower.sph.harvard.edu/metaphlan2) can be used with shotgun data to identify microbial
community composition. For more information on how to run MetaPhlAn2, see the [MetaPhlAn2 tutorial](https://bitbucket.org/biobakery/biobakery/wiki/metaphlan2).

In [4]:
# get the names of the metaphlan2 output files based on the input files
taxon_profiles = [file_basename+"_metaphlan2_taxonomy.tsv" for file_basename in output_file_basenames]
metaphlan2_sam = [file_basename+"_metaphlan2_alignments.sam" for file_basename in output_file_basenames]

# add tasks to the workflow to run metaphlan2 on all input files
for shotgun_file, profile, sam in zip(input_files, taxon_profiles, metaphlan2_sam):
    workflow.add_task(
        "metaphlan2.py [depends[0]] --input_type [args[0]] --output_file [targets[0]] --samout [targets[1]] --no_map",
        depends=shotgun_file,
        targets=[profile,sam],
        args=input_extension)

**Step 5:** Add tasks to run HUMAnN2 on all of the files in the input folder using the MetaPhlAn2 profiles. 
[HUMAnN2](http://huttenhower.sph.harvard.edu/humann2) (the HMP Unified Metabolic Analysis Network) is a tool for 
identifying metabolic pathways in metagenomic and metatranscriptomic sequencing data. For more information on how to run 
HUMAnn2, see the [HUMAnN2 tutorial](https://bitbucket.org/biobakery/biobakery/wiki/humann2).

In [5]:
# get the name of the expected humann2 output files based on the input files
genefam=[file_basename+"_genefamilies.tsv" for file_basename in output_file_basenames]
pathabund=[file_basename+"_pathabundance.tsv" for file_basename in output_file_basenames]
pathcov=[file_basename+"_pathcoverage.tsv" for file_basename in output_file_basenames]

# add tasks to the workflow to run humann2 on all input files
# provide the metaphlan2 output as input (optional as humann2 can also compute the profile using metaphlan2)
for shotgun_file, profile, genes, pathab, pathc in zip(input_files, taxon_profiles, genefam, pathabund, pathcov):
    workflow.add_task(
        "humann2 --input [depends[0]] --taxonomic-profile [depends[1]] --output [args[0]]",
        depends=[shotgun_file, profile],
        targets=[genes, pathab, pathc],
        args=os.path.dirname(genes))

**Step 6:** Add tasks to run StrainPhlAn using the MetaPhlAn2 output files as input. [StrainPhlAn](http://segatalab.cibio.unitn.it/tools/strainphlan/) identifies single
nucleotide polymorphisms in unique marker genes to analyze shotgun sequencing data at a strain-level resolution. For 
more information on how to run StrainPhlAn2, see the [StrainPhlan tutorial](https://bitbucket.org/biobakery/biobakery/wiki/strainphlan).

In [6]:
# get the names of the marker output files
strainphlan_markers = [file_basename+"_metaphlan2_alignments.markers" for file_basename in output_file_basenames]

# create files of markers for each of the input files
for sam, markers in zip(metaphlan2_sam,strainphlan_markers):
    workflow.add_task(
        "sample2markers.py --ifn_samples [depends[0]] --input_type sam --output_dir [args[0]]",
        depends=sam,
        targets=markers,
        args=os.path.dirname(markers)) 

# add tasks to run strainphlan, using options (ie marker in clade) to allow for demo input files
# notice for this add task we capture the task that is returned, this is optional
# tasks can be provided as dependencies for other tasks in a workflow
options=" --marker_in_clade 0.2 --keep_alignment_files"
strainphlan_task=workflow.add_task(
    "strainphlan.py --ifn_samples [args[0]]/*.markers --ifn_markers [depends[0]] "+\
    "--ifn_ref_genomes [depends[1]] --output_dir [args[0]] --clades s__Eubacterium_siraeum "+options,
    depends=[strainphlan_e_siraeum_markers,strainphlan_e_siraeum_reference]+strainphlan_markers,
    targets=os.path.join(output_folder,"RAxML_bestTree.s__Eubacterium_siraeum.tree"),
    args=os.path.dirname(strainphlan_markers[0]))

**Step 7:** Add PanPhlAn tasks to the workflow. [PanPhlAn](http://segatalab.cibio.unitn.it/tools/panphlan/) is a tool for strain identification and tracking along with functional analysis. 
For more information on how to run PanPhlAn, see the [PanPhlAn tutorial](https://bitbucket.org/CibioCM/panphlan/wiki/Tutorial).

In [7]:
# add the panphlan tasks after getting the names of the output files
panphlan_genes = [file_basename+"_"+panphlan_species+".csv.bz2"  for file_basename in output_file_basenames]

# map against species database
# set small read length to allow for demo input files
for fasta, gene in zip(input_files, panphlan_genes):
    workflow.add_task(
        "panphlan_map.py -c [args[0]] -i [depends[0]] -o [targets[0]] --i_bowtie2_indexes [args[1]] --readLength 25",
        depends=fasta,
        targets=gene,
        args=[panphlan_species,panphlan_db_folder])

# create gene family presence absence profile
# use very sensitive thresholds for demo data
panphlan_task=workflow.add_task(
    "panphlan_profile.py -c [args[0]] -i [args[1]] --o_dna [targets[0]] --i_bowtie2_indexes [args[2]] "+\
    "--min_coverage 1 --left_max 1.70 --right_min 0.30 --verbose",
     depends=panphlan_genes,
     targets=os.path.join(output_folder,"panphlan_gene_matrix.tsv"),
     args=[panphlan_species,os.path.dirname(panphlan_genes[0]),panphlan_db_folder])

**Step 8:** Run the workflow with six tasks running in parallel.

In [8]:
workflow.go(jobs=6)

(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task 32: panphlan_map.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task 10: metaphlan2.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task 31: panphlan_map.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task  8: metaphlan2.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task 30: panphlan_map.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task  6: metaphlan2.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task 29: panphlan_map.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task  4: metaphlan2.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task 28: panphlan_map.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task  2: metaphlan2.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task 27: panphlan_map.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Ready    ** Task  0: metaphlan2.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] **Started  ** Task 32: panphlan_map.py
(Jun 06 11:37:02) [ 0/26 -   0.00%] *